# Bounded Knapsack

The Bounded Knapsack quantum kata is a series of exercises designed to teach you to use Grover search to solve the knapsack problem. 

* Learn more about the knapsack problem in the [Wikipedia article](https://en.wikipedia.org/wiki/Knapsack_problem).
* [SolveSATWithGrover](./../SolveSATWithGrover/SolveSATWithGrover.ipynb) and [GraphColoring](./../GraphColoring/GraphColoring.ipynb) are the other katas covering oracle implementation for solving constraint satisfaction problems. This kata is more challenging, so you might want to try these two first.

The kata covers the following topics:
 - writing operations to perform arithmetic tasks.
 - writing oracles to implement constraints based on:
  - the 0-1 knapsack problem, a simple version of the knapsack problem;
  - the bounded knapsack problem.
 - using the oracle with Grover's algorithm to solve the knapsack decision problem.
 - using Grover's algorithm repeatedly to solve the knapsack optimization problem.

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with the `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl+Enter (⌘+Enter on macOS).

Within each section, tasks are given in approximate order of increasing difficulty.

## Part I. 0-1 Knapsack Problem

You are given $n$ items, indexed $0$ to $n-1$. The item with index $i$ has a weight of $w_i$ and yields a profit of $p_i$. In the original 0-1 knapsack decision problem, we wish to determine whether we can put a subset of items in a knapsack to get a total profit of at least $P$, while not exceeding a maximum weight $W$.

However, we will slightly modify the problem: the total profit must be *strictly greater than* $P$, rather than at least $P$.

In the following tasks you will implement an oracle that evaluates whether a particular register of $n$ qubits, representing an item combination, satisfies the described conditions.

### Task 1.1. Read combination from a register

**Input:** An array of $n$ qubits, which are guaranteed to be in one of the $2^n$ basis states.  

**Output:** The item combination that this state represents, expressed as a boolean array of length $n$. The $i$-th element of the array should be `true` (indicating that $i$-th item is selected) if and only if the $i$-th qubit of the register is in the $|1\rangle$ state. The operation should not change the state of the qubits.
     
**Example:** For $n = 3$ and the qubits state $|101\rangle$, return `[true, false, true]`.

In [2]:
%kata T11_MeasureCombination01

open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Measurement;

operation MeasureCombination01 (selectedItems : Qubit[]) : Bool[] {
    mutable output = ConstantArray(Length(selectedItems), false);
    
    let results = MultiM(selectedItems);
    for i in 0 .. Length(results) - 1 {
        set output w/= i <- ResultAsBool(results[i]);
    }
    
    return output;
}

Success!

### Task 1.2. Calculate the number of (qu)bits necessary to hold the maximum total value

**Input:** An array of $n$ positive integers, describing the value (the weight or the profit) of each item.

**Output:** The minimum number of (qu)bits needed to store the maximum total value of the items.

**Example:** For $n = 4$ and `itemValues = [1, 2, 3, 4]`, the maximum total value is $10$, which requires at least $4$ qubits to store it, so $4$ is returned.

In [3]:
function Plus(a : Int, b : Int) : Int {
    return a + b;
}

Plus

In [4]:
%kata T12_NumBitsTotalValue01

open Microsoft.Quantum.Math;

function NumBitsTotalValue01 (itemValues : Int[]) : Int {
    let sum = Fold(Plus, 0, itemValues);
    return Floor(Lg(IntAsDouble(sum)) + 1.0);
}

Success!

In [5]:
function First(tuple : (Int, Bool)) : Int {
    let (first, second) = tuple;
    return first;
} 

First

### Task 1.3. Calculate the total value of selected items

**Inputs:**
     1) An array of $n$ integers, describing the values (the weights or the profits) of the items.
     2) An array of $n$ qubits, describing whether each item is put into the knapsack.
     3) An array of qubits `total` in the $|0...0\rangle$ state to store the total value of the selected items. 
It is guaranteed that there are enough qubits to store the total value.

**Goal:** Transform the `total` array to represent, in little-endian format, the sum of the values of the items that are put in the knapsack. The input qubits can be in a superposition state. Leave the qubits in selectedItems in the same state they started in.

**Example:** For $n = 4$ and `itemValues = [1, 2, 3, 4]`, the input state $|1001\rangle|0000\rangle$ should be transformed to $|1001\rangle|1010\rangle$, since items $0$ and $3$ are put in the knapsack, and `itemValues[0] + itemValues[3] = 1 + 4 = 5 = 1010₂`.

In [6]:
function Second(tuple : (Int, Bool)) : Bool {
    let (first, second) = tuple;
    return second;
} 

Second

In [7]:
%kata T13_CalculateTotalValueOfSelectedItems01

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Arrays;

operation CalculateTotalValueOfSelectedItems01 (
    itemValues : Int[], 
    selectedItems : Qubit[], 
    total : Qubit[]
) : Unit is Adj+Ctl {
    // Each qubit in selectedItems determines whether the corresponding value is added.
    // Adding the selected items is implemented using a library operation with a control from each qubit of the selectedItems.
    let totalLE = LittleEndian(total);
    for (control, value) in Zipped(selectedItems, itemValues) {
        Controlled IncrementByInteger([control], (value, totalLE));
    }
}

Success!

### Task 1.4. Compare an integer stored in a qubit array with an integer (>)

**Inputs:**
1. An array of `D` qubits representing an integer in little-endian format.
2. An integer $b$ ($0 \le b \le 2^D - 1$).
3. A qubit in an arbitrary state (target qubit).
     
**Goal:** Flip the state of the target qubit if the integer represented by the qubit array is greater than $b$. The input qubits can be in superposition. Leave the qubits in the qubit array in the same state they started in.

**Example:** For $b = 11$, the input state $|1011\rangle|0\rangle$ should be transformed to $|1011\rangle|1\rangle$, since $1101_2 = 13_{10} > 11_{10}$.

In [8]:
%kata T14_CompareQubitArrayGreaterThanInt

operation CompareQubitArrayGreaterThanInt (qs : Qubit[], b : Int, target : Qubit) : Unit is Adj+Ctl {
    let D = Length(qs);

    // Convert b into array of bits in little endian format
    let binaryB = IntAsBoolArray(b, D);

    // Iterate descending from the most significant digit (stored last), flipping the target qubit
    // upon finding i such that qs[i] > binaryB[i], AND qs[j] = binaryB[j] for all j > i.
    // The X gate flips qs[i] to represent whether qs[i] and binaryB[i] are equal, to
    // be used as controls for the Toffoli.
    // Thus, the Toffoli will only flip the target when qs[i] = 1, binaryB[i] = 0, and  
    // qs[j] = 1 for all j > i (meaning a and binaryB have the same digits above i).

    for i in D - 1 .. -1 .. 0 {
        if (not binaryB[i]) {
            // Checks if qs has a greater bit than b at index i AND all bits above index i have equal values in a and b.
            Controlled X(qs[i ...], target);
            // Flips the qubit if b's corresponding bit is 0.
            // This temporarily sets the qubit to 1 if the corresponding bits are equal.
            X(qs[i]);
        }
    }

    // Uncompute
    ApplyPauliFromBitString(PauliX, false, binaryB, qs);
}

Success!

### Task 1.5. Compare an integer stored in a qubit array with an integer (≤)

**Inputs:**
1. An array of D qubits representing an integer in little-endian format.
2. An integer $b$ ($0 \le b \le 2^D - 1$).
3. A qubit in an arbitrary state (target qubit).

**Goal:** Flip the state of the target qubit if the integer represented by the qubit array is less than or equal to $b$. The input qubits can be in superposition. Leave the qubits in the qubit array in the same state they started in.

**Example:** For `b = 7`, the input state $|1010\rangle|0\rangle$ should be transformed to $|1010\rangle|1\rangle$, since $0101_2 = 5_{10} ≤ 7_{10}$.

In [9]:
%kata T15_CompareQubitArrayLeqThanInt

operation CompareQubitArrayLeqThanInt (qs : Qubit[], b : Int, target : Qubit) : Unit is Adj+Ctl {
    CompareQubitArrayGreaterThanInt(qs, b, target);
    X(target);
}

Success!

### Task 1.6. Verify that the total weight doesn't exceed the limit $W$
**Inputs:**
1. An integer $W$, the maximum weight the knapsack can hold.
2. An array of $n$ integers, describing the weights of the items: `itemWeights[i] = wᵢ`.
3. An array of $n$ qubits, describing whether each item is put into the knapsack.
4. A qubit in an arbitrary state (target qubit).

**Goal:** Flip the state of the target qubit if the total weight is less than or equal to $W$. The input qubits can be in superposition. Leave the qubits in the qubit array in the same state they started in.

In [10]:
%kata T16_VerifyTotalWeight01

operation VerifyTotalWeight01 (W : Int, itemWeights : Int[], selectedItems : Qubit[], target : Qubit) : Unit is Adj+Ctl {
   let N = NumBitsTotalValue01 (itemWeights);
   use total = Qubit[N];
   within {
       CalculateTotalValueOfSelectedItems01 (itemWeights, selectedItems, total);
   }
   apply {
       CompareQubitArrayLeqThanInt(total, W, target);
   }
}

Success!

### Task 1.7. Verify that the total profit exceeds the threshold $P$
**Inputs:**
1. An integer $P$, the threshold which the total profit must exceed.
2. An array of $n$ integers, describing the profits of the items: `itemProfits[i] = pᵢ`.
3. An array of $n$ qubits, describing whether each item is put into the knapsack.
4. A qubit in an arbitrary state (target qubit).

**Goal:** Flip the state of the target qubit if the total profit is greater than $P$. The input qubits can be in superposition. Leave the qubits in the qubit array in the same state they started in.

In [11]:
%kata T17_VerifyTotalProfit01

operation VerifyTotalProfit01 (P : Int, itemProfits : Int[], selectedItems : Qubit[], target : Qubit) : Unit is Adj+Ctl {
   let N = NumBitsTotalValue01 (itemProfits);
   use total = Qubit[N];
   within {
       CalculateTotalValueOfSelectedItems01 (itemProfits, selectedItems, total);
   }
   apply {
       CompareQubitArrayGreaterThanInt(total, P, target);
   }
}

Success!

### Task 1.8. Verify the solution to the 0-1 knapsack problem

**Inputs:**
1. An integer $W$, the maximum weight the knapsack can hold.
2. An integer $P$, the threshold which the total profit must exceed.
3. An array of $n$ integers, describing the weights of the items: `itemWeights[i] = wᵢ`.
4. An array of $n$ integers, describing the profits of the items: `itemProfits[i] = pᵢ`.
5. An array of $n$ qubits, describing whether each item is put into the knapsack.
6. A qubit in an arbitrary state (target qubit).

**Goal:** Flip the state of the target qubit if the selection of the items in selectedItems satisfies both constraints:
* the total weight of all items is less than or equal to $W$, and
* the total profit of all items is greater than $P$.

The input qubits can be in superposition. Leave the qubits in the qubit array in the same state they started in.

In [12]:
%kata T18_VerifyKnapsackProblemSolution01

operation VerifyKnapsackProblemSolution01 (
    W : Int, P : Int, 
    itemWeights : Int[], 
    itemProfits : Int[], 
    selectedItems : Qubit[], 
    target : Qubit
) : Unit is Adj+Ctl {
    use anc = Qubit[2];
    within {
        VerifyTotalWeight01 (W, itemWeights, selectedItems, anc[0]);
        VerifyTotalProfit01 (P, itemProfits, selectedItems, anc[1]);
    }
    apply {
        Controlled X(anc, target);
    }
}

Success!

## Part II. Bounded Knapsack Problem

In this version of the problem we still consider $n$ items, indexed $0$ to $n-1$, the item with index $i$ has a weight of $w_i$ and yields a profit of $p_i$. 
But in the bounded knapsack version of the problem, unlike in the 0-1 knapsack problem, each type of item can have more than one copy, and can be selected multiple times. 
Specifically, there are $b_i$ copies of item type $i$ available, so this item type can be selected between $0$ and $b_i$ times, inclusive.
Let $x_i$ represent the number of copies of index $i$ that are put into the knapsack; the constraint $0 ≤ x_i ≤ b_i$ must hold for all $i$. 
Thus, we seek a combination `xs = [x₀, x₁, ..., xₙ₋₁]` which has total weight at most $W$, and has total profit that is strictly greater than $P$.

The comparators from Part I (tasks 1.4-1.5) can be reused, but the operations for calculating total weight and profit will need to be rewritten.

### Task 2.1. Read combination from a jagged array of qubits

**Input:** A jagged array of qubits of length $n$. Array `selectedItemCounts[i]` represents the binary notation of $x_i$ in little-endian format. Each qubit is guaranteed to be in one of the basis states ($|0\rangle$ or $|1\rangle$).

**Output:** An integer array of length $n$, containing the combination that this jagged array represents. The $i$-th element of the output array should have value $x_i$. The operation should not change the state of the qubits.

**Example:** For state `selectedItemCounts = [|101⟩, |1110⟩, |0100⟩]`, return `[5, 7, 2]`.

In [13]:
%kata T21_MeasureCombination

operation MeasureCombination (selectedItemCounts : Qubit[][]) : Int[] {
    mutable output = ConstantArray<Int>(Length(selectedItemCounts), 0);
    for i in 0 .. Length(selectedItemCounts) - 1{
        let results = MultiM(selectedItemCounts[i]);
        set output w/= i <- ResultArrayAsInt (results);
    }
    
    return output;
}

Success!

### Task 2.2. Convert an array into a jagged array

<span style="color:red"><b>This task is temporarily not available in Notebook format; please use Q# project version of the BoundedKnapsack kata to complete it.</b></span>

To simplify access to the register as an array of integers within the oracle, we reorganize the register into several qubit arrays that represent, in little-endian format,  the number of copies of each item type. We can do the same transformation with arrays of other types, for example, arrays of classical bits (boolean or integer) that store binary notations of several integers. To make our code reusable, we can make it type-parameterized.

**Inputs:**
1. An array of type `T` (`T` could be qubits or classical bits).
2. An array of $n$ integers $b_i$.

**Output:** A jagged array of n arrays of type `T`. $i$-th element of the return value should have enough bits to represent the integer $b_i$ in binary notation. The length of the input array of `T` is defined to be able to store all integers $b_i$.

**Example:** For `b = [6, 15, 13]` and a register of qubits in state $|10111100100\rangle$, you need to use $3$, $4$, and $4$ bits to represent the integers $b_i$, so you'll return an array of qubit arrays `[|101⟩, |1110⟩, |0100⟩]`.

In [26]:
function RegisterAsJaggedArray<'T> (array : 'T[], b : Int[]) : 'T[][] {
    // Identify bit lengths of integers bᵢ.
    let bitLengths = Mapped(BitSizeI, b);
    // Partition the array in accordance to these lengths.
    // Remember to discard the last element in the return of Partitioned, 
    // which will be empty, since our partitioning is precise.
    return Most(Partitioned(bitLengths, array));
}

RegisterAsJaggedArray

### Task 2.3. Verification of limits satisfaction

**Inputs:**
1. An array of $n$ integers `itemCountLimits[i] = bᵢ`.
2. A jagged array of $n$ qubits. Each array `selectedItemCounts[i]` represents the number of items $x_i$ in little-endian format.
3. A qubit in an arbitrary state (target qubit).

**Goal:** Flip the state of the target qubit if $x_i ≤ b_i$ for all $i$. The input qubits can be in superposition. Leave the qubits in qubit array in the same state they started in.

In [15]:
%kata T23_VerifyLimits

operation VerifyLimits (itemCountLimits : Int[], selectedItemCounts : Qubit[][], target : Qubit) : Unit is Adj+Ctl {
    let N = Length(itemCountLimits);
    use anc = Qubit[N];
    
    within {
        for i in 0 .. N - 1 {
            CompareQubitArrayLeqThanInt (selectedItemCounts[i], itemCountLimits[i], anc[i]);
        }
    }
    apply {
        Controlled X (anc, target);
    }
}

Success!

### Task 2.4. Increment a quantum integer by a product of classical and quantum integers

**Inputs:**
1. An integer $x$.
2. An array of $n$ qubits, representing an arbitrary integer $y$.
3. An array of $m$ qubits, representing an arbitrary integer $z$.

**Goal:** Increment register $z$ by the product of $x$ and $y$. Perform the increment modulo $2^m$ (you don't need to track the carry bit). The input qubits can be in superposition. Leave the qubits in register $y$ in the same state they started in.

In [17]:
%kata T24_IncrementByProduct

operation IncrementByProduct (x : Int, y : Qubit[], z : Qubit[]) : Unit is Adj+Ctl {
    // x * y = y + y + ... y (x times)
    //       = x + x + ... x (y times)
    let totalZ = LittleEndian(z);
    for i in 0 .. Length(y) - 1 {
        Controlled IncrementByInteger([y[i]], (x * (2^i), totalZ));
    }
}

Success!

### Task 2.5. Calculate the number of (qu)bits necessary to hold the maximum total value

**Inputs:**
1. An array of $n$ positive integers, describing the value (the weight or the profit) of each item.
2. An array of $n$ positive integers, describing the $b_i$ - the limits on the maximum number of items of type $i$ that can be selected.

**Output:** The minimum number of (qu)bits needed to store the maximum total value of the items.

**Example:** For `n = 4`, `itemValues = [1, 2, 3, 4]`, and `itemCountLimits = [2, 5, 4, 3]`, the maximum possible total value is  $1 \cdot 2 + 2 \cdot 5 + 3 \cdot 4 + 4 \cdot 3 = 36$, which requires at least $6$ qubits, so $6$ is returned.

In [20]:
function PlusMul(a : Int, tuple : (Int, Int)) : Int {
    let (val, amt) = tuple;
    return a + (val * amt);
}

PlusMul

In [21]:
%kata T25_NumBitsTotalValue

function NumBitsTotalValue (itemValues : Int[], itemCountLimits : Int[]) : Int {
    let zipped = Zipped(itemValues, itemCountLimits);
    let sum = Fold(PlusMul, 0, zipped);
    
    return Floor(Lg(IntAsDouble(sum)) + 1.0);
}

Success!

### Task 2.6. Calculate total value of selected items

**Inputs:**
1. An array of $n$ integers, describing the values (the weights or the profits) of the items.
2. A jagged array of qubits of length $n$. Array `selectedItems[i]` represents the number of items of type $i x_i$ in little-endian format.
3. An array of qubits `total` in the $|0...0\rangle$ state to store the total value of the selected items. It is guaranteed that there are enough qubits to store the total value.

**Goal:** Transform the `total` array to represent, in little-endian format, the sum of the values of the items that are put in the knapsack. The input qubits can be in a superposition state. Leave the qubits in `selectedCounts` in the same state they started in.

In [22]:
%kata T26_CalculateTotalValueOfSelectedItems

operation CalculateTotalValueOfSelectedItems (
    itemValues : Int[], 
    selectedCounts : Qubit[][], 
    total : Qubit[]
) : Unit is Adj+Ctl {
    let N = Length(itemValues);
    for i in 0 .. N - 1 {
        IncrementByProduct (itemValues[i], selectedCounts[i], total);
    }
}

Success!

### Task 2.7. Verify that the total weight doesn't exceed the limit $W$

**Inputs:**
1. An integer $W$, the maximum weight the knapsack can hold.
2. An array of $n$ integers, describing the weights of the items: `itemWeights[i] = wᵢ`.
3. An array of $n$ positive integers, describing the $b_i$ - the limits on the maximum number of items of type $i$ that can be selected.
4. A jagged array of qubits, describing the number of each item type put into the knapsack. Array `selectedCounts[i]` represents $x_i$ in little-endian format.
5. A qubit in an arbitrary state (target qubit).

**Goal:** Flip the state of the target qubit if the total weight is less than or equal to $W$. The input qubits can be in superposition. Leave the qubits in the `selectedCounts` array in the same state they started in.

In [23]:
%kata T27_VerifyTotalWeight

operation VerifyTotalWeight (
    W : Int, 
    itemWeights : Int[], 
    itemCountLimits : Int[], 
    selectedCounts : Qubit[][], 
    target : Qubit
) : Unit is Adj+Ctl {
    let N = NumBitsTotalValue (itemWeights, itemCountLimits);
    use total = Qubit[N];
    within {
        CalculateTotalValueOfSelectedItems (itemWeights, selectedCounts, total);
    }
    apply {
        CompareQubitArrayLeqThanInt (total, W, target);
    }
}

Success!

### Task 2.8. Verify that the total profit exceeds threshold $P$

**Inputs:**
1. An integer $P$, the threshold which the total profit must exceed.
2. An array of $n$ integers, describing the profits of the items: `itemProfits[i] = pᵢ`.
3. An array of $n$ positive integers, describing the $b_i$ - the limits on the maximum number of items of type $i$ that can be selected.
4. A jagged array of qubits, describing the number of each item type put into the knapsack. Array `selectedCounts[i]` represents $x_i$ in little-endian format.
5. A qubit in an arbitrary state (target qubit).

**Goal:** Flip the state of the target qubit if the total profit is greater than $P$. The input qubits can be in superposition. Leave the qubits in the `selectedCounts` array in the same state they started in.

In [25]:
%kata T28_VerifyTotalProfit

operation VerifyTotalProfit (
    P : Int, 
    itemProfits : Int[], 
    itemCountLimits : Int[], 
    selectedCounts : Qubit[][], 
    target : Qubit
) : Unit is Adj+Ctl {
    let N = NumBitsTotalValue (itemProfits, itemCountLimits);
    use total = Qubit[N];
    within {
        CalculateTotalValueOfSelectedItems (itemProfits, selectedCounts, total);
    }
    apply {
        CompareQubitArrayGreaterThanInt (total, P, target);
    }
}

Success!

### Task 2.9. Verify the solution to the bounded knapsack problem

**Inputs:**
1. An integer $W$, the maximum weight the knapsack can hold.
2. An integer $P$, the threshold which the total profit must exceed.
3. An array of $n$ integers, describing the weights of the items: `itemWeights[i] = wᵢ`.
4. An array of $n$ integers, describing the profits of the items: `itemProfits[i] = pᵢ`.
5. An array of $n$ integers, describing the maximum numbers of each type of item that can be selected: `itemCountLimits[i] = bᵢ`.
6. An array of $Q$ qubits, describing the numbers of each type of item selected for the knapsack. ($Q$ is the minimum number of qubits necessary to store a concatenation of the numbers up to $b_i$.)
7. A qubit in an arbitrary state (target qubit).

**Goal:** Flip the state of the target qubit if the selection of the items in `selectedCountsRegister` satisfies both constraints:
* the total weight of all items is less than or equal to $W$, 
* the total profit of all items is greater than $P$, and
* the number of $i$-th type of item is less than or equal to $b_i$.

The input qubits can be in superposition. Leave the qubits in register in the same state they started in.

In [29]:
%kata T29_VerifyKnapsackProblemSolution

operation VerifyKnapsackProblemSolution (
    W : Int, P : Int, 
    itemWeights : Int[], 
    itemProfits : Int[], 
    itemCountLimits : Int[], 
    selectedCountsRegister : Qubit[], 
    target : Qubit
) : Unit is Adj+Ctl {
    let selectedCountsRegisterJagged = RegisterAsJaggedArray(selectedCountsRegister, itemCountLimits);

    use (outputW, outputP, outputB) = (Qubit(), Qubit(), Qubit());
    
    within {
        VerifyTotalWeight (W, itemWeights, itemCountLimits, selectedCountsRegisterJagged, outputW);
        VerifyTotalProfit (P, itemProfits, itemCountLimits, selectedCountsRegisterJagged, outputP);
        VerifyLimits (itemCountLimits, selectedCountsRegisterJagged, outputB);
    }
    apply {
        Controlled X([outputW, outputP, outputB], target);
    }
}

Success!

## Part III. Using Grover's algorithm for knapsack optimization problems

Coming soon...